In [2]:
from time import monotonic

import lsdb
from dask.distributed import Client
from tape import Ensemble, ColumnMapper

In [3]:
%%time

objects = lsdb.read_hipscat('/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14',
                            columns=['ps1_objid', 'ra', 'dec', 'nobs_g', 'nobs_r', 'nobs_i',
                                     'Norder', 'Dir', 'Npix'])
sources = lsdb.read_hipscat('/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source',
                            columns=['ps1_objid', 'mjd', 'mag', 'magerr', 'catflags', 'band',
                                     'Norder', 'Dir', 'Npix'])

CPU times: user 1min 11s, sys: 19.6 s, total: 1min 30s
Wall time: 2min 40s


In [4]:
%%time
joined_sources = objects.join(
    sources,
    left_on='ps1_objid',
    right_on='ps1_objid',
    output_catalog_name='ztf_axs_sources'
)

CPU times: user 4min 18s, sys: 10.4 s, total: 4min 29s
Wall time: 4min 29s


In [5]:
joined_sources

,ps1_objid_ztf_dr14,ra_ztf_dr14,dec_ztf_dr14,nobs_g_ztf_dr14,nobs_r_ztf_dr14,nobs_i_ztf_dr14,Norder_ztf_dr14,Dir_ztf_dr14,Npix_ztf_dr14,ps1_objid_ztf_source,mjd_ztf_source,mag_ztf_source,magerr_ztf_source,catflags_ztf_source,band_ztf_source,Norder_ztf_source,Dir_ztf_source,Npix_ztf_source
npartitions=311037,,,,,,,,,,,,,,,,,,
0,int64,float64,float64,int32,int32,int32,int32,int32,int32,int64,float64,float32,float32,int16,string,int32,int32,int32
281474976710656,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13834987686537986048,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18446744073709551615,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
client = Client(n_workers=4, memory_limit='16GB')
client

/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46221 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46221/status,
Dashboard: http://127.0.0.1:46221/status,Workers: 4
Total threads: 96,Total memory: 59.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38538,Workers: 4
Dashboard: http://127.0.0.1:46221/status,Total threads: 96
Started: Just now,Total memory: 59.60 GiB
Comm: tcp://127.0.0.1:41426,Total threads: 24
Dashboard: http://127.0.0.1:37070/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:35195,


In [13]:
%%time

ens = Ensemble(client)
column_mapper = ColumnMapper(
    id_col='_hipscat_index',
    time_col='mjd_ztf_source',
    flux_col='mag_ztf_source',
    err_col='magerr_ztf_source',
    band_col='band_ztf_source',
)
ens.from_dask_dataframe(
    object_frame=objects._ddf,
    source_frame=joined_sources._ddf,
    sorted=True,
    sort=False,
    sync_tables=False,
    column_mapper=column_mapper,
)

ens.source.query('catflags_ztf_source == 0 & magerr_ztf_source > 0').update_ensemble()
ens = ens.calc_nobs(by_band=False, label="nobs", temporary=False)
max_nobs = ens.object['nobs_total'].max()

/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/dask/dataframe/core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/dask/dataframe/core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


CPU times: user 4.32 s, sys: 442 ms, total: 4.76 s
Wall time: 4.69 s


In [14]:
%%time

t = monotonic()
result = max_nobs.compute()
dt = monotonic() - t

/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 135.50 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2023-12-14 03:42:51,300 - distributed.worker.memory - WARNING - Worker is at 93% memory usage. Pausing worker.  Process memory: 14.00 GiB -- Worker memory limit: 14.90 GiB
2023-12-14 03:42:52,493 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:38726 (pid=129233) exceeded 95% memory budget. Restarting...
2023-12-14 03:42:55,051 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:38726
Traceback (most recent call last):
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/tornado/iostream.py", line 1116

    convert_stream_closed_error(self, e)
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/distributed/comm/tcp.py", line 141, in convert_stream_closed_error
    raise CommClosedError(f"in {obj}: {exc.__class__.__name__}: {exc}") from exc
distributed.comm.core.CommClosedError: in <TCP (closed)  local=tcp://127.0.0.1:42883 remote=tcp://127.0.0.1:41864>: BrokenPipeError: [Errno 32] Broken pipe
2023-12-14 04:14:17,680 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:41426
Traceback (most recent call last):
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was th

KeyboardInterrupt: 

2023-12-14 04:14:18,164 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('repartition-split-7f7d892d6dd561a487d9b59ca1559856', 292229))" coro=<Worker.execute() done, defined at /astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/distributed/worker_state_machine.py:3610>> ended with CancelledError
2023-12-14 04:14:18,165 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('repartition-split-7f7d892d6dd561a487d9b59ca1559856', 292227))" coro=<Worker.execute() done, defined at /astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/distributed/worker_state_machine.py:3610>> ended with CancelledError
2023-12-14 04:14:18,165 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('repartition-split-7f7d892d6dd561a487d9b59ca1559856', 292219))" coro=<Worker.execute() done, defined at /astro/users/kmalanch/.virtualenvs/tape/li

In [15]:
with open('result.txt', 'w') as f:
    f.write(f'{dt = }\n{result = }\n')

NameError: name 'dt' is not defined

2023-12-14 04:14:18,497 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:42883 -> tcp://127.0.0.1:38967
Traceback (most recent call last):
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/distributed/worker.py", line 1783, in get_data
    response = await comm.read(deserializers=serializers)
  File "/astro/users/kmalanch/.virtualenvs/tape/lib/python3.9/site-packages/distributed/comm/tcp.py", line 240, in read
    convert_stream_closed_err

In [11]:
client.close()